# TRABALHO PRÁTICO 3 - GRUPO 14

## Exercício 1 - Problema 1 - DILITHIUM

Este problema consistia em implementar a técnica **DILITHIUM** que é um esquema de assinatura digital presente no concurso NIST PQC e usa o esquema LWE básico como ponto de partida. Nesta implementação do Dilithium foi utilizado os passos presentados no paper [Dilithium](https://www.dropbox.com/sh/5wxso1sx6gnnym0/AACF7Kvmrkpqmv9oCfL4jq6na/Dilithium/dilithium/Supporting_Documentation/dilithium_nist.pdf?dl=0) 

### **IMPORTS**

In [10]:
from math import *
from cryptography.hazmat.primitives import hashes
from pickle import load, dumps
import random as rn
import os, sys
import numpy as np

### **RESOLUÇÃO DO PROBLEMA**

Na classe abaixo é implementado o algoritmo DILITHIUM que irá gerar uma assinatura utilizando uma chave privada, sendo a chave pública utilizada para verificar a autenbticidade da assinatura. Assim sendo, será necessário implementar 3 funcionalidades principais:

**Geração do par de chaves:** A função `Gen` tem como objetivo gerar o par de chaves a ser utilizado para a assinatura da mensagem e para a verificação da assinatura. Para isso começamos por gerar um valor aleatório de 32 bytes, $\zeta$, que será utilizado pela função de hash `tripleH` para gerar as variáveis  $\rho, \varsigma$ e K. A variável $\varsigma$ será utilizada na função `vectorGen` para gerar os vetores $\textbf{s}_1$ e $\textbf{s}_2$ $\in S_\eta^l \times S_\eta^k$, já a variavél $\rho$ será utilizada pela função `ExpandA` para gerar a matriz $\textbf{A} \in R_q^{k \times l}$.

* **Geração das Samples $\textbf{s}_1$ e $\textbf{s}_2$:** Estes dois vetores serão gerados apartir da condição $||w|| < \tau, w \in \{\textbf{s}_1,\textbf{s}_2\}$, sendo que para isso é necessário que todos os elementos $x$ de $\textbf{s}_1$ e $\textbf{s}_2$ seja gerados segundo a condição $x = ||n||_\infty = |n\ mod^\pm\ q|, n \in \varsigma$.
* **Geração da matriz A:** Para gerarmos a matriz A começamos por gerar um inteiro de 2 bytes em formato *little endian* apartir da expressão $n * i + j, i \in \{0...k\}$ e $j \in \{0...l\}$. Estes 2 bytes vão ser utilizados juntamente coma a variável $\rho$ na função de hash `genHash128`, sendo o output utilizado pela função `genInteger`. Esta função começa por utilizar os 3 primeiros bytes do output recebido, transformando o bit mais alto do terceiro byte a zero. Este 3 bytes serão interpretados como um inteiro em formato *little endian*, gerando-se assim os coeficentes dos polinómios da matriz A.

De seguida geramos o vetor $t$ através da expressão $t = A * \textbf{s}_1 +  \textbf{s}_2$ que será utilizado na função `Power2Round` que tem como objetivo partir *bit wise* os elementos de $R_q$ devolvendo assim o par $(t_1, t_0)$. A variável $t_1$ será utilizada na função de hash `CRH` juntamento com a variável $\rho$ para gerar a variável $tr$. No final a chave pública é dada por $pk = (\rho, t_1)$, já a chave privada será dada por $sk = (\rho, K, tr, \textbf{s}_1,\textbf{s}_2, t_0)$

**Geração da assinatura:** A função `Sign` tem como objetivo assinar uma mensagem a ser enviada, devolvendo a assinatura gerada. Para isso, utilizamos a chave privada $sk$ e a mensagem em bytes $M$. Iniciamos a função com a geração da matriz A utilizando a função *ExpandA* e a variavél $\rho$. De seguida, geramos a variável $\mu$ através da função *CRH* utilizando as variáveis $tr$ e $M$ e a variável $\rho'$ através da função *CRH* e das variáveis $K$ e $\mu$. Ao longo de um ciclo while vamos gerando as variáveis $z$ e $h$ de forma a que estam respeitem a condição: 
$||\textbf{z}||_\infty < \gamma_1-\beta$ **and** $||\textbf{r}_0||_\infty < \gamma_2-\beta$ **and** $||c * \textbf{t}_0||_\infty < \gamma_2$ **and** # of $1$'s in $\textbf{h} \le \omega$. Para isso começamos por gerar a variável $y$ através da função `ExpandMask` utilizando as variáveis $\rho'$ e $k$, com $k$ inicialmente igual a 0.

* **Geração do vetor $y$:** Para gerarmos o vetor $y$ começamos por gerar um inteiro de 2 bytes em formato *little endian* apartir da expressão $k + i, i \in \{0...l\}$. Estes 2 bytes vão ser utilizados juntamente coma a variável $\rho'$ na função de hash `genHash256`, sendo o output utilizado pela função `genIntegerMask`. Esta função começa por utilizar os 3 primeiros bytes do output recebido, transformando os dois 2 bits ou os 4 bits do terceiro byte dependendo se o $\gamma_1$ é igual a $2^{17}$ ou $2^{19}$, respetivamente. Este 3 bytes serão interpretados como um inteiro em formato *little endian*, gerando-se assim os coeficentes dos polinómios do vetor $y$.

De seguida, geramos o vetor $w$ através da expressão $w = A * y$ sendo este vetor utilizado na função `HighBits` que tem como objetivo extrair os *high-order bits* do output da função `Decompose`. Esta última tem como objetivo reduzir o tamanho da chave pública através da geração dos *higher-order* and *lower-order bits* dos elementos pertencentes a $Z_q$. Assim, geramos a variável $\textbf{w}_1$ que será utilizada, juntamente com a variável $\mu$, na função de hash `H` para assim calcular a variável $\tilde{c}$ . O output desta função será utilizado pela função `SampleInBall` para gerar o vetor $c$.

* **Geração do vetor $c$:** Para gerarmos o vetor $c$ começamos por utilizar a função `genHash256` para assim gerar uma hash de $n$ bytes apartir da variável recebida como parâmetro. De seguida, geramos os *sign bits* que podem ser obtidos utilizando a função `genSignBits`. Esta função utiliza os primeiros 8 bytes, sendo que os primeiros $\tau$ (peso de $c$) bits vão corresponder ao *sign bits*. Cada byte do output de *genHash256* vai corresponder a um inteiro em formato *little endian* que será gerado pela função `genIntegerJ`. Sendo *j* os valores inteiros gerados e *s* os *signs bits*, de seguida será aplicado o algoritmo *sample in ball* tal como se encontra na página 10 da documentação.

No final, a variavél $z$ irá corresponder à expressão $z = y + c * \textbf{s}_1$ e a variável $r_0$ irá ser obtida através da função `LowBits` que tem como objetivo extrair os *lower-order bits* do output da função *Decompose*. A variavél $h$ será obtida através da função `MakeHint` que tem como objetivo calcular um *hint* que será usado para recuperar high-order bits da soma $\textbf{w}-c\textbf{s}_2+c\textbf{t}_0$. Caso a condição apresentada em cima não se verificar, então o ciclo começa novamente com $k = k + l$. Caso a condição se verificar, então a assinatura é dada por $\sigma = (z,h,\tilde{c})$.


**Verificação da assinatura:** A função `Verify` tem como objetivo verificar a autenticidade da assinatura $\sigma$ recebida como parâmetro quando associada à mensagem $M$ utilizando para isso a a chave pública $pk$. Para isso começamos por determinar a matriz A através da função *ExpandA* e da variavél $\rho$. De seguida calculamos a variável $y = CRH(CRH(\rho\ ||\ \textbf{t}_1)\ ||\ M)$. Para calcularmos a variável $c$ utilizamos a função *SampleInBall* e a variável $\tilde{c}$. Estas variáveis todas serão utilizadas na função `UseHint` para assim determinar $\textbf{w}_1' :=$ UseHint$(\textbf{h},\textbf{Az}-c\textbf{t}_1 \cdot 2^d,2\gamma_2)$. Esta função tem como objetivo usar o *hint* para recuperar *high-order bits* da soma apresentada. Para verificar a autenticidade da assinatura verificamos a condição $||\textbf{z}||_\infty<\gamma_1-\beta$ **and** $\tilde{c} =$ H$(\mu\ ||\ \textbf{w}_1')$ **and** # of $1$'s in $\textbf{h} \le \omega$


#### Notas:
As funções *Decompose*, *UseHint* e *MakeHint* foram implementas de forma a devolverem e receberem vetores para assim ir de encontro com os tipos de dados apresentados nos restantes passos do algoritmo. É também necessário ter emk atenção que a função *Power2Round*, entre outras, recebe como parâmetro vetores e que para isso utiliza as funções `modVector` e `modpmVector` que aplica a operação de $mod$ e $mod^\pm$, respetivamente, para cada elemento do vetor de polinómios. A função `modpm` tem como objetivo implementar a operação $mod^\pm$, segundo se encontra descrito na página 9 da documentação. 


In [11]:
class DILITHIUM:
    
    def __init__(self):
        self.q,self.n,self.d,self.c,self.y1,self.y2,self.k,self.l,self.nn,self.w,self.b,self.R,self.Rq = self.setup()
    
    #Parâmetros da técnica DILITHIUM - NIST level 5 - 5+
    def setup(self):
        
        q = 8380417 
        n = 256
        d = 13
        c = 60
        y1 = 2**19 
        y2 = (q - 1)//32
        k = 9
        l = 8
        nn = 2
        b = 120
        w = 85
        
        Z.<x> = ZZ[]
        R.<x> = QuotientRing(Z,Z.ideal(x^n+1))

        Zq.<x> = GF(q)[]
        fi     = x^n + 1
        Rq.<x> = QuotientRing(Zq,Zq.ideal(fi))
        
        return q,n,d,c,y1,y2,k,l,nn,w,b,R,Rq
    
    # ------------------------------------- FUNÇÕES AUXLIARES ---------------------------------------- #
    
    #Implementação do mod+- segundo "Modular Reductions" - pag 9 
    def modpm(self,r,a):
        
        if a%2 == 0:
            limit = a/2
        else:
            limit = (a-1)/2
            
        rMod = r % a
        
        if rMod > limit:
            rMod -= a
            
        return rMod
    
    # Implementação do mod para todos os elementos de um vetor
    def modVector(self, v,a):
        
        rAux = []
        
        for i in v:
            iAux = []
            for j in i:
                iAux.append(mod(j,a))
                
            rAux.append(self.R(iAux))
            
        return vector(rAux)
    
    # Implementação do mod+- para todos os elementos de um vetor
    def modpmVector(self, v,a):
        
        rAux = []
        
        for i in v:
            iAux = []
            for j in i:
                iAux.append(self.modpm(j,a))
                
            rAux.append(self.R(iAux))
            
        return vector(rAux)
    
    #Implementação de ||v||∞ segundo "Sizes of elements" - pag 9
    def normInf(self, v):
        
        maxsV = []
        
        # ||v||∞ = max(||vi||∞)
        for vi in v:
            maxsVi = []
            
            #||vi||∞ = max(||i||∞)
            for i in vi:
                #||i||∞ = |i mod ± q|
                maxsVi.append(self.modpm(i,self.q))
                
            maxsV.append(max(maxsVi))
        
        return max(maxsV)
    
    #Geração das samples s1 e s2 - pag 9
    def vectorGen(self,c):
        
        # We will write Sη to denote all elements w ∈ R such that ||w||∞ ≤ η.
        
        hashS = self.H(c, self.l*self.n + self.k*self.n)
        index = 0
        s1 = []
        
        for i in range(self.l):
            poli = [] 
            
            for j in range(self.n):
                num = hashS[index]
                index += 1
                
                poli.append(self.modpm(num,self.nn))
                
            s1.append(self.R(poli))
            
        s2 = []
        
        for i in range(self.k):
            poli = [] 
            
            for j in range(self.n):
                num = hashS[index]
                index += 1
                
                poli.append(self.modpm(num,self.nn))
                
            s2.append(self.R(poli))
        
        return (vector(s1),vector(s2))
    
    #Implementação da Função H para geração do triplo (ρ, ς, K)
    def tripleH(self, C):
        
        #256*3 bits = 96 bytes
        digest = hashes.Hash(hashes.SHAKE256(int(96))) 
        digest.update(C)
        buffer = digest.finalize() 
        
        return (buffer[:32], buffer[32:64], buffer[64:])
    
    #Função auxiliar que permite transformar a matrix A de Rq para R
    def fromRqToR(self, A):
        
        mtx = []
        
        for row in A:
            
            newRow = []
            
            for elem in row:
                
                newRow.append(self.R(elem))
                
            mtx.append(newRow)
            
        return Matrix(mtx)
        
    # ------------------------------------- ALGORITMOS DE SUPORTE (PAG 12)  ------------------------------------ #
    
    #Função que parte elementos de Rq Bit-wise - pag 11-12
    def Power2Round(self,r,d):

        r = self.modVector(r,self.q)
        r0 = self.modpmVector(r,2^d)
        
        return ((r - r0)/2^d, r0)
    
    #Função que reduz o tamanho da chave publica através da geração dos “higher-order” and “lower-order” bits 
    # dos elementos pertencentes a Zq - Aplicada a vetores
    def Decompose(self,r,alpha):
        
        r = self.modVector(r,self.q)
        r0 = self.modpmVector(r,alpha)
        
        rElems = []
        r0Elems = []
        
        for ri, r0i in zip(r, r0):
            
            r1Coefs = []
            r0Coefs = []

            for rij, r0ij in zip(ri, r0i):
                
                if (rij - r0ij == self.q - 1):
                    r1ij = 0
                    r0ij = r0ij-1
                else:
                    r1ij = (rij - r0ij)/alpha
                    
                r1Coefs.append(r1ij)
                r0Coefs.append(r0ij)
                
            rElems.append(self.R(r1Coefs))
            r0Elems.append(self.R(r0Coefs))

        return (vector(rElems),vector(r0Elems))
    
    #Função que extrai os “higher-order” bits do output de Decompose
    def HighBits(self,r,alpha):
        
        (r1,r0) = self.Decompose(r,alpha)
        
        return r1
    
    #Função que extrai os “lower-order” bits do output de Decompose
    def LowBits(self,r,alpha):
        
        (r1,r0) = self.Decompose(r,alpha)
        
        return r0
    
    #Função que calcula um hint que será usado para recuperar high-order bits da soma - pag 11 e 12
    # Aplicada para vetores
    def MakeHint(self,z,r,alpha):
        
        r1 = self.HighBits(r,alpha)
        v1 = self.HighBits(r + z,alpha)
        
        vec = []
        for r1i, v1i in zip(r1, v1):
            for r1ij, v1ij in zip(r1i, v1i):
                vec.append(r1ij != v1ij)
        
        return vec
    
    #Função que usa o hint para recuperar high-order bits da soma - pag 11 e 12
    # Aplicada para vetores
    def UseHint(self,h,r,alpha):
        
        m = (self.q - 1)/alpha
        (r1,r0) = self.Decompose(r,alpha)
        
        rAux = []
        i = 0
        
        for r0i, r1i in zip(r0,r1):
            elem = []
            
            for r0ij, r1ij in zip(r0i,r1i):
                if h[i] :
                    if r0ij > 0:
                        elem.append(mod((r1ij + 1),m))
                        
                    else:
                         elem.append(mod((r1ij - 1),m))
                else:
                    elem.append(r1ij)
                    
                i += 1
                
            rAux.append(self.R(elem))
        
        return vector(rAux)
    
    # ----------------------------------------------- FUNÇÕES HASH  -------------------------------------------- #
    
    #Implementação da função H - pag 12
    def H(self, m, length):
        
        digest = hashes.Hash(hashes.SHAKE256(int(length)))
        digest.update(m)
        h = digest.finalize()
    
        return h
    
    #Implementação da "Collision resistant hash" - pag 20
    def CRH(self,seed):
        
        prf = hashes.Hash(hashes.SHAKE256(int(48)))
        prf.update(seed)
        
        return prf.finalize()
    
    # -------------------------------------------- FUNÇÕES DE SAMPLING  ----------------------------------------- #
    
    #Implementação da função "Expanding the Matrix A" - pag 19
    def ExpandA(self,ro):
        
        #Função Hash a ser utilizada neste contexto
        def genHash128(seed):
            
            digest = hashes.Hash(hashes.SHAKE128(int(self.n-1)))
            digest.update(seed)
            
            return digest.finalize()
        
        #Gera o integer segundo as regras apresentadas
        def genInteger(offset, seedShake):
            
            if(len(offset)<3):
                offset = seedShake
                
            threeBytes = bytearray(offset[:3])
            threeBytes[2]  &= 0x7f
            threeBytes = bytes(threeBytes)
            
            number = int.from_bytes(threeBytes, "little")
            
            return number, offset[3:]
            
        A = []
        
        for i in range(self.k):
            row = []
            for j in range(self.l):
                coefs = []
                intTwoBytes = int(self.n*i + j).to_bytes(2, "little")
                hashShake = genHash128(ro + intTwoBytes)
                seedShake = hashShake
                
                for cof in range(self.n): #rejection sampling??
                    number, hashShake = genInteger(hashShake, seedShake)
                
                    coefs.append(number)
                row.append(self.Rq(coefs))
            A.append(row)
            
        return Matrix(A)
    
    #Implementação da função "Hashing to a Ball" - pag 19
    def SampleInBall(self,ro):
        
        #Função Hash a ser utilizada neste contexto
        def genHash256(seed):
            
            digest = hashes.Hash(hashes.SHAKE256(int(self.n)))
            digest.update(seed)
            
            return digest.finalize()
        
        #Função que retira dos primeiros 8 bytes os sign bits
        def genSignBits(seed):
            
            digest = int.from_bytes(seed[:8], 'little')
            
            bits = [int(digit) for digit in list(ZZ(digest).binary())]
    
            return bits[:self.c], seed[8:]
            
        #Função que gera o valor de j
        def genIntegerJ(offset, seedShake):
            
            if(len(offset)<1):
                offset = seedShake
                
            number = int.from_bytes(bytes(offset[0]), "little")
            
            return number, offset[1:]

        c = [0]*self.n
        hashShake = genHash256(ro)
        seedShake = hashShake
        
        signBits, hashShake = genSignBits(hashShake)
        
        for i in range(self.n - self.c, self.n): #rejection sampling??
            
            j, hashShake = genIntegerJ(hashShake,seedShake)
            s = signBits[i-self.n + self.c]
            c[i] = c[j]
            c[j] = (-1)^s
            
        return self.R(c)
    
    #Implementação da função "Sampling the vectors" - pag 20
    def ExpandMask(self, ro, k):
        
        #Função Hash a ser utilizada neste contexto
        def genHash256(seed):
            
            digest = hashes.Hash(hashes.SHAKE256(int(self.n-1)))
            digest.update(seed)
            
            return digest.finalize()
        
        #Função que gera o inteiro segundo regras apresentadas
        def genIntegerMask(offset, seedShake):
            
            if(len(offset)<3):
                offset = seedShake
                
            threeBytes = bytearray(offset[:3])
            
            if self.y1 == 2**17:
                threeBytes[2]  &= 0x3
            else:
                threeBytes[2]  &= 0xf
        
            threeBytes = bytes(threeBytes)
            
            number = int.from_bytes(threeBytes, "little")
            
            return number, offset[3:]

        y = []
        
        for i in range(self.l):
            coefs = []
            intTwoBytes = int(k + i).to_bytes(2, "little")
            hashShake = genHash256(ro + intTwoBytes)
            seedShake = hashShake
                
            for cof in range(self.n):
                number, hashShake = genIntegerMask(hashShake, seedShake)
                
                coefs.append(number - (self.y1 -1))
            y.append(self.R(coefs))
            
        return vector(y)
    
    # ---------------------------------------- FUNÇÕES PRINCIPAIS (PAG 13)  ------------------------------------- #
        
    #FUNÇÃO: Gera o par de chaves (publica e privada) para a assinatura e verificação
    def Gen(self):
        
        # ζ ← {0, 1}^256
        C = os.urandom(32)
        
        #(ρ, ς, K ) ∈ {0, 1}^256×3 := H(ζ)
        (ro, c, K) = self.tripleH(C)
        
        #(s1,s2) ∈ Sη^l × Sη^k := H(ς)
        (s1,s2) = self.vectorGen(c)
        
        #A ∈ Rq^k×l := ExpandA(ρ)
        A = self.ExpandA(ro)
        
        #t := A*s1 + s2
        t = A * s1 + s2
        
        #(t1,t0) := Power2Round(t, d)
        (t1, t0) = self.Power2Round(t,self.d)
        
        #tr ∈ {0, 1}^384 := CRH(ρ || t1)
        tr = self.CRH(ro + dumps(t1))
        
        return (ro, t1),(ro, K, tr, s1, s2, t0) #pk, sk
    
    #FUNÇÃO: Assina a mensagem a enviar M utilizando a chave privada sk
    def Sign(self, sk, M):
        
        (ro, K, tr, s1, s2, t0) = sk
        
        #A ∈ Rq^k×l := ExpandA(ρ)
        A = self.ExpandA(ro)
        
        #μ ∈ {0, 1}^384:= CRH(tr || M)
        yy = self.CRH(tr + M)
        
        #κ := 0
        k = 0
        
        #(z, h) := ⊥
        (z,h) = (None, None)
        
        #ρ' ∈ {0,1}^384 := CRH(K || μ)
        roNew = self.CRH(K + yy)
        
        while z == None or h == None:
            
            #y ∈ Sγ^l := ExpandMask(ρ', κ)
            y = self.ExpandMask(roNew, k)
            
            #w := Ay
            w = A * y
            
            #w1 := HighBits(w, 2*γ2 )
            w1 = self.HighBits(w, 2 * self.y2)
            
            #c' ∈ {0,1}^256 := H(μ || w1)
            cAux = self.H(yy + dumps(w1), 32)
            #c ∈ Bτ := SampleInBall(c')
            c = self.SampleInBall(cAux)
            
            #z := y + cs 1
            z = y + c * s1
            
            #r0 := LowBits(w − c*s2 , 2*γ2)
            r0 = self.LowBits(w - c * s2, 2 * self.y2)
            
            
            #||z||∞ ≥ γ1 − β or ||r0||∞ ≥ γ2 − β
            if self.normInf(z) >= (self.y1 - self.b) and self.normInf(r0) >= (self.y2 - self.b):
                
                #(z, h) := ⊥
                (z,h) = (None, None)
                
            else:
                
                #h := MakeHint(−c*t0 , w − c*s2 + c*t0 , 2*γ2)
                h = self.MakeHint( -c * t0, w - c * s2 + c * t0, 2 * self.y2)
                
                #||c*t0||∞ ≥ γ2 or the # of 1’s in h is greater than ω
                if self.normInf(c * t0) >= self.y2 or h.count(True) > self.w:
                    
                    #(z, h) := ⊥
                    (z,h) = (None, None)
            
            #κ := κ + l
            k = k + self.l
                
        return (z,h,cAux) #sigma
    
    #FUNÇÃO: Verifica a assinatura na mensagem utilizando a chave publica pk
    def Verify(self,pk,M,sigma):
        
        (ro, t1) = pk
        (z,h,cAux) = sigma
        
        #A ∈ Rq^k×l := ExpandA(ρ)
        A = self.ExpandA(ro)
        
        #μ ∈ {0, 1}^384 := CRH(CRH(ρ + t1) + M )
        y = self.CRH(self.CRH(ro + dumps(t1)) + M)
        
        #c := SampleInBall(c')
        c = self.SampleInBall(cAux)
        
        #w' = UseHint(h, A*z − c*t1 * 2^d , 2*γ2)
        w1 = self.UseHint(h, (self.fromRqToR(A) * z) - (c * t1 * 2^self.d), 2 * self.y2)
        
        #||z||∞ < γ1 − β and c' = H(μ || w') and J# of 1’s in h is ≤ ω
        return self.normInf(z) < (self.y1-self.b) and cAux == self.H(y + dumps(w1), 32) and h.count(True) <= self.w

#### **CENÁRIO DE TESTE**

In [12]:
dlth = DILITHIUM()

message = "Mensagem a ser assinada"
messageErrada = "Mensagem a ser assinado"

public, private = dlth.Gen()

signature = dlth.Sign(private, message.encode())

print("SIGNATURE: ")
print(signature)

result = dlth.Verify(public, message.encode(), signature)

print("MENSAGEM VÁLIDA")
print(message)

if(result):
    print("Assinatura Válida!")
else:
    print("Assinatura Inválida!")
    
result2 = dlth.Verify(public, messageErrada.encode(), signature)

print("MENSAGEM INVÁLIDA")
print(messageErrada)

if(result2):
    print("Assinatura Válida!")
else:
    print("Assinatura Inválida!")

SIGNATURE: 
((-249504*x^255 - 475517*x^254 + 217354*x^253 - 234626*x^252 - 110688*x^251 - 48290*x^250 - 165675*x^249 + 319970*x^248 + 259210*x^247 - 243678*x^246 + 349165*x^245 - 25471*x^244 + 152009*x^243 + 91610*x^242 + 1985*x^241 + 184334*x^240 - 141553*x^239 - 148266*x^238 - 148670*x^237 - 215122*x^236 - 190700*x^235 - 3391*x^234 + 165014*x^233 + 522999*x^232 + 471830*x^231 - 343405*x^230 + 25150*x^229 + 206208*x^228 + 107982*x^227 + 521126*x^226 + 508948*x^225 + 176728*x^224 + 261323*x^223 + 343830*x^222 + 462364*x^221 + 306977*x^220 + 442765*x^219 + 162741*x^218 - 514059*x^217 + 272503*x^216 + 168724*x^215 - 6404*x^214 + 331803*x^213 + 451487*x^212 - 467475*x^211 + 114059*x^210 + 23940*x^209 + 189118*x^208 + 230715*x^207 - 284342*x^206 - 258016*x^205 - 149396*x^204 - 460883*x^203 + 498964*x^202 + 43637*x^201 - 455107*x^200 + 387854*x^199 + 445249*x^198 + 300294*x^197 - 419104*x^196 - 283384*x^195 - 159769*x^194 + 214683*x^193 - 398496*x^192 + 252773*x^191 + 18013*x^190 - 222009*x